In [41]:
import pandas as pd

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 435.7 kB/s eta 0:00:03
   - -------------------------------------- 0.0/1.2 MB 435.7 kB/s eta 0:00:03
   - -------------------------------------- 0.0/1.2 MB 435.7 kB/s eta 0:00:03
   - -------------------------------------- 0.0/1.2 MB 435.7 kB/s eta 0:00:03
   - -------------------------------------- 0.0/1.2 MB 178.6 kB/s eta 0:00:07
   -- ------------------------------------- 0.1/1.2 MB 262.6 kB/s eta 0:00:05
   --- ------------------------------------ 0.1/1.2 MB 309.1 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.2 MB 309.1 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.2 MB 309.1 kB/s eta 0:00:04
   ---- ----------------------------------- 0.1/1.2 MB 277.4 kB/s eta 0:00:04
   ------ --------------------------------- 0.2/1.2 MB 372.0 kB/s eta 0:00:03
   ------- -------------------------------- 0.2/1.2 MB 444.3 kB/s eta 0:0

In [ ]:
#Подключение к БД и вывод содержимого таблицы 'flights'

In [54]:
import psycopg2
engine=psycopg2.connect("dbname='demo' user='postgres' host='localhost' port='5432' password='123'")
df=pd.read_sql('select * from flights', con=engine)
df

C:\Users\#slash\AppData\Local\Temp\ipykernel_1328\4009591350.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql('select * from flights', con=engine)


,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,182,PG0402,2017-09-01 09:25:00+00:00,2017-09-01 10:20:00+00:00,DME,LED,Scheduled,321,NaT,NaT
1,1996,PG0335,2017-08-26 06:30:00+00:00,2017-08-26 08:35:00+00:00,DME,JOK,Scheduled,CN1,NaT,NaT
2,5979,PG0384,2017-08-26 09:10:00+00:00,2017-08-26 09:40:00+00:00,DME,BZK,Scheduled,SU9,NaT,NaT
3,8136,PG0138,2017-08-28 07:15:00+00:00,2017-08-28 08:20:00+00:00,VKO,RTW,Scheduled,CR2,NaT,NaT
4,10455,PG0277,2017-09-12 08:45:00+00:00,2017-09-12 12:10:00+00:00,SVO,OVB,Scheduled,773,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...
65659,65660,PG0063,2017-06-26 16:25:00+00:00,2017-06-26 17:10:00+00:00,SKX,SVO,Arrived,CR2,2017-06-26 16:27:00+00:00,2017-06-26 17:12:00+00:00
65660,65661,PG0063,2017-08-10 16:25:00+00:00,2017-08-10 17:10:00+00:00,SKX,SVO,Arrived,CR2,2017-08-10 16:27:00+00:00,2017-08-10 17:13:00+00:00
65661,65662,PG0063,2017-06-16 16:25:00+00:00,2017-06-16 17:10:00+00:00,SKX,SVO,Arrived,CR2,2017-06-16 16:28:00+00:00,2017-06-16 17:14:00+00:00
65662,65663,PG0063,2017-09-10 16:25:00+00:00,2017-09-10 17:10:00+00:00,SKX,SVO,Scheduled,CR2,NaT,NaT


In [1]:
import numpy as n

In [ ]:
#Запрос к БД. Вывод информации по загрузке рейсов

In [61]:
import psycopg2
query= f"""
WITH commers_load AS
(SELECT 
fl.flight_id,
fl.flight_no,
fl.scheduled_departure,
fl.departure_airport,
fl.arrival_airport,
fl.aircraft_code,
count(tf.ticket_no) AS fact,
	(SELECT 
	   COUNT(st.seat_no)
	   FROM seats AS st
	   WHERE st.aircraft_code=fl.aircraft_code) AS total_seats

FROM flights AS fl
INNER JOIN ticket_flights AS tf
ON fl.flight_id=tf.flight_id
WHERE fl.status = 'Arrived'
GROUP BY 1, 2, 3, 4)
SELECT 
commers_load.flight_id,
commers_load.flight_no,
commers_load.scheduled_departure,
commers_load.departure_airport,
commers_load.arrival_airport,
commers_load.fact,
commers_load.total_seats,
ad.model,
ROUND (commers_load.fact::numeric/commers_load.total_seats::numeric, 2) AS fraction,
case when ROUND(commers_load.fact::numeric/commers_load.total_seats::numeric, 2) < 0.10 then 'полет убыточный'
else 'в пределах нормы'
end AS status
FROM commers_load 
INNER JOIN aircrafts_data AS ad
ON commers_load.aircraft_code=ad.aircraft_code
ORDER BY commers_load.scheduled_departure"""
engine=psycopg2.connect("dbname='demo' user='postgres' host='localhost' port='5432' password='123'")
df=pd.read_sql(query, con=engine)

C:\Users\#slash\AppData\Local\Temp\ipykernel_10400\1431805813.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query, con=engine)


In [62]:
#Вывод первых 15-ти строк

In [97]:
df.head(15)

,flight_id,flight_no,scheduled_departure,departure_airport,arrival_airport,fact,total_seats,model,fraction,status
0,52986,PG0328,2017-05-16 23:45:00+00:00,VVO,KHV,39,97,"{'en': 'Sukhoi Superjet-100', 'ru': 'Сухой Суп...",0.40,в пределах нормы
1,31219,PG0244,2017-05-17 00:20:00+00:00,YKS,LED,5,116,"{'en': 'Airbus A319-100', 'ru': 'Аэробус A319-...",0.04,полет убыточный
2,52831,PG0660,2017-05-17 00:55:00+00:00,VVO,IKT,1,50,"{'en': 'Bombardier CRJ-200', 'ru': 'Бомбардье ...",0.02,полет убыточный
3,49972,PG0199,2017-05-17 01:35:00+00:00,IKT,LED,12,170,"{'en': 'Airbus A321-200', 'ru': 'Аэробус A321-...",0.07,полет убыточный
4,52072,PG0357,2017-05-17 02:05:00+00:00,KHV,LED,24,222,"{'en': 'Boeing 767-300', 'ru': 'Боинг 767-300'}",0.11,в пределах нормы
5,34706,PG0135,2017-05-17 02:25:00+00:00,BTK,DME,8,116,"{'en': 'Airbus A319-100', 'ru': 'Аэробус A319-...",0.07,полет убыточный
6,38894,PG0391,2017-05-17 02:40:00+00:00,MJZ,OVB,4,50,"{'en': 'Bombardier CRJ-200', 'ru': 'Бомбардье ...",0.08,полет убыточный
7,53723,PG0592,2017-05-17 03:00:00+00:00,TOF,SVO,8,97,"{'en': 'Sukhoi Superjet-100', 'ru': 'Сухой Суп...",0.08,полет убыточный
8,50542,PG0659,2017-05-17 03:30:00+00:00,IKT,VVO,4,50,"{'en': 'Bombardier CRJ-200', 'ru': 'Бомбардье ...",0.08,полет убыточный
9,39244,PG0035,2017-05-17 03:35:00+00:00,MJZ,CNN,2,12,"{'en': 'Cessna 208 Caravan', 'ru': 'Сессна 208...",0.17,в пределах нормы


In [67]:
# вывод типов данных

In [77]:
type (df)

pandas.core.frame.DataFrame

In [70]:
df.dtypes

flight_id                            int64
flight_no                           object
scheduled_departure    datetime64[ns, UTC]
departure_airport                   object
arrival_airport                     object
fact                                 int64
total_seats                          int64
model                               object
fraction                           float64
status                              object
dtype: object

In [ ]:
#число уникальных значений по столбцу "model"

In [117]:
b=df['model'].value_counts()
print(b)

model
{'en': 'Bombardier CRJ-200', 'ru': 'Бомбардье CRJ-200'}       11627
{'en': 'Sukhoi Superjet-100', 'ru': 'Сухой Суперджет-100'}    10576
{'en': 'Cessna 208 Caravan', 'ru': 'Сессна 208 Караван'}       4130
{'en': 'Airbus A321-200', 'ru': 'Аэробус A321-200'}            1962
{'en': 'Boeing 767-300', 'ru': 'Боинг 767-300'}                1800
{'en': 'Boeing 737-300', 'ru': 'Боинг 737-300'}                1759
{'en': 'Airbus A319-100', 'ru': 'Аэробус A319-100'}            1625
{'en': 'Boeing 777-300', 'ru': 'Боинг 777-300'}                 907
Name: count, dtype: int64


In [ ]:
#число повторяющихся значений по столбцам

In [149]:
df1=df.apply(lambda x:sum(x.duplicated()))
print(df1)

flight_id                  0
flight_no              33916
scheduled_departure    20410
departure_airport      34292
arrival_airport        34292
fact                   34029
total_seats            34378
model                  34378
fraction               34285
status                 34384
dtype: int64


In [ ]:
#минимальное значение по столбцу 'total_seats(int64)'

In [173]:
value_min=df['total_seats'].min()
print('минимальное кол-во мест в самолете:', value_min)

минимальное кол-во мест в самолете: 12


In [ ]:
#максимальное значение количества мест в самолете(столбец 'total_seats')

In [175]:
value_max=df['total_seats'].max()
print('максимальное кол-во мест в самолете:', value_max)

максимальное кол-во мест в самолете: 402


In [ ]:
#максимальное значение по загрузке самолета, где 1.0-100%

In [171]:
value_max=df['fraction'].max()
print('максимальное значение:', value_max)

максимальное значение: 1.0


In [ ]:
#подсчет медианы по загрузке рейсов(столбец 'fraction')

In [195]:
med=df['fraction'].median()
print(med)

0.58


In [ ]:
#подсчет среднего значения загрузки рейсов (столбец 'fraction')

In [213]:
sred=df['fraction'].mean()
sred


0.5616823707322747